In [23]:
from __future__ import print_function
import tensorflow as tf
from six.moves import cPickle as pickle
import numpy as np

In [2]:
pickle_file = 'data1/notMNIST.pickle'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels



train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [32]:
graph = tf.Graph()


In [33]:
batch_size = 128

In [34]:
with graph.as_default():
    tf_train_dataset = tf.placeholder(np.float32,shape=([batch_size,image_size * image_size]))
    tf_train_label = tf.placeholder(np.float32,shape=([batch_size,num_labels]))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables
    
    
    
    # droupout probability (set default as 1.0, so evaluation time, it's turned off)
    dropout_prob = tf.placeholder_with_default(1.0, shape=())
    #Layer 1 
    weights_layer_1  = tf.Variable(tf.truncated_normal([image_size * image_size, 100]))
    bias_layer_1 = tf.Variable(tf.zeros(100))
    relu_out_bfr_1 = tf.nn.relu(tf.matmul(tf_train_dataset,weights_layer_1)+bias_layer_1)
    #first dropout
    relu_out_1 = tf.nn.dropout(relu_out_bfr_1,dropout_prob)
    
    
    #Layer2
    
    weights_layer_2 = tf.Variable(tf.truncated_normal([100,num_labels]))
    bias_layer_2 = tf.Variable(tf.zeros(num_labels))
    logit_bfr = tf.matmul(relu_out_1,weights_layer_2)+bias_layer_2
    logit=tf.nn.dropout(logit_bfr,dropout_prob)
    
    #On loss we'll introduce l2 regularizer..
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_label,logits=logit)+ (0.01 * tf.nn.l2_loss(weights_layer_1)) +(0.01 * tf.nn.l2_loss(weights_layer_2) ) )
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logit)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_layer_1)+bias_layer_1), weights_layer_2) + bias_layer_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_layer_1)+bias_layer_1), weights_layer_2) + bias_layer_2)

In [36]:
num_steps=30000
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized..!!!')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_dataset.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset: (offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data,tf_train_label:batch_labels}
        if (step % 500 == 0):
            feed_dict = {tf_train_dataset:batch_data,tf_train_label:batch_labels,dropout_prob:0.5} # <- see the dropout pred set/reset
        
        _,l,preds = session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f',(step,l))
            print('Minibatch accuracy at step %d : %.1f%%',(step,accuracy(preds,batch_labels)))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized..!!!
Minibatch loss at step %d: %f (0, 522.09869)
Minibatch accuracy at step %d : %.1f%% (0, 7.8125)
Validation accuracy: 22.3%
Test accuracy: 23.3%
Minibatch loss at step %d: %f (500, 4.3059816)
Minibatch accuracy at step %d : %.1f%% (500, 46.09375)
Validation accuracy: 79.5%
Test accuracy: 85.9%
Minibatch loss at step %d: %f (1000, 2.1976495)
Minibatch accuracy at step %d : %.1f%% (1000, 46.875)
Validation accuracy: 80.4%
Test accuracy: 86.1%
Minibatch loss at step %d: %f (1500, 2.2665772)
Minibatch accuracy at step %d : %.1f%% (1500, 44.53125)
Validation accuracy: 81.7%
Test accuracy: 88.0%
Minibatch loss at step %d: %f (2000, 2.392252)
Minibatch accuracy at step %d : %.1f%% (2000, 35.9375)
Validation accuracy: 79.4%
Test accuracy: 86.3%
Minibatch loss at step %d: %f (2500, 1.9400361)
Minibatch accuracy at step %d : %.1f%% (2500, 49.21875)
Validation accuracy: 81.1%
Test accuracy: 87.0%
Minibatch loss at step %d: %f (3000, 2.5648081)
Minibatch accuracy at step %d : %.1f%

Minibatch loss at step %d: %f (27000, 1.820554)
Minibatch accuracy at step %d : %.1f%% (27000, 50.0)
Validation accuracy: 81.8%
Test accuracy: 87.8%
Minibatch loss at step %d: %f (27500, 1.8177397)
Minibatch accuracy at step %d : %.1f%% (27500, 50.78125)
Validation accuracy: 79.1%
Test accuracy: 85.0%
Minibatch loss at step %d: %f (28000, 1.7054706)
Minibatch accuracy at step %d : %.1f%% (28000, 48.4375)
Validation accuracy: 82.9%
Test accuracy: 89.2%
Minibatch loss at step %d: %f (28500, 2.333246)
Minibatch accuracy at step %d : %.1f%% (28500, 38.28125)
Validation accuracy: 81.1%
Test accuracy: 87.3%
Minibatch loss at step %d: %f (29000, 2.1495185)
Minibatch accuracy at step %d : %.1f%% (29000, 36.71875)
Validation accuracy: 79.8%
Test accuracy: 86.2%
Minibatch loss at step %d: %f (29500, 1.7740273)
Minibatch accuracy at step %d : %.1f%% (29500, 49.21875)
Validation accuracy: 83.2%
Test accuracy: 89.4%
